In [ ]:
# Copyright 2025 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Agent Engine in Vertex AI - orchestration

## Init Vertex AI SDK

In [ ]:
%pip install --upgrade --user --quiet google-cloud-aiplatform[agent_engines,adk,langchain,ag2,llama_index]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 678.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.7/153.7 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.2/734.2 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#### Note: <font color="red">Need to restart the kernel after installing</font>

In [ ]:
# Note:  Need to restart the kernel
%pip install --upgrade --user --quiet langchain-google-vertexai

In [ ]:
# @title Authentication to access to GCP

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# @title Define constants
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
MODEL_NAME = "gemini-2.0-flash"
BUCKET_URI = f"gs://agent-0417"

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

## Configuration and initialization

### Initialize Vertex AI

In [ ]:
# @title Create a bucket.
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://agent-0319/...
ServiceException: 409 A Cloud Storage bucket named 'agent-0319' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [ ]:
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Build a LangGraph Agent.

In [ ]:
#@title Customize orchestration with LangGraph.

# Use runnable interface of Langchain.
# https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/develop/langgraph#orchestration
# https://python.langchain.com/v0.2/docs/concepts/#runnable-interface

from vertexai import agent_engines

def langgraph_builder(*, model, **kwargs):
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser
    from langgraph.graph import END, MessageGraph

    output_parser = StrOutputParser()

    planner = ChatPromptTemplate.from_template(
        "Generate an argument about: {input}"
    ) | model | output_parser

    pros = ChatPromptTemplate.from_template(
        "List the positive aspects of {input}"
    ) | model | output_parser

    cons = ChatPromptTemplate.from_template(
        "List the negative aspects of {input}"
    ) | model | output_parser

    summary = ChatPromptTemplate.from_template(
        "Input:{input}\nGenerate a final response given the critique",
    ) | model | output_parser

    builder = MessageGraph()
    builder.add_node("planner", planner)
    builder.add_node("pros", pros)
    builder.add_node("cons", cons)
    builder.add_node("summary", summary)

    builder.add_edge("planner", "pros")
    builder.add_edge("planner", "cons")
    builder.add_edge("pros", "summary")
    builder.add_edge("cons", "summary")
    builder.add_edge("summary", END)
    builder.set_entry_point("planner")

    return builder.compile()


In [ ]:
  #@title Crate a tool for Google search.
  from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

  google_search_tool = Tool(
      google_search = GoogleSearch()
  )

In [ ]:
#@title Build a Langgraph Agent.

from vertexai import agent_engines

agent = agent_engines.LanggraphAgent(
    model=MODEL_NAME,
    tools=[google_search_tool],
    runnable_builder=langgraph_builder,
)

In [ ]:
#@title Query to an agent.
query = "생성형 AI의 대해서 설명해주세요."

response = agent.query(input={"role": "user", "content": query})

for re in response:
  display(Markdown((re['kwargs']['content'])))
  print('----------------------------------------------------------')


생성형 AI의 대해서 설명해주세요.

------------------------------------------------------------


## 생성형 AI의 발전과 윤리적 고려: 창조적 가능성과 책임의 균형

생성형 AI는 텍스트, 이미지, 오디오, 비디오 등 다양한 형태의 콘텐츠를 스스로 생성해내는 혁신적인 기술입니다. 과거 단순히 데이터를 분석하고 예측하는 데 그쳤던 AI 기술과는 달리, 생성형 AI는 주어진 데이터 패턴을 학습하여 완전히 새로운 콘텐츠를 창조해낼 수 있다는 점에서 엄청난 잠재력을 지니고 있습니다. 이러한 발전은 예술, 디자인, 콘텐츠 제작, 연구 개발 등 다양한 분야에서 혁신을 가져올 가능성을 제시하며, 인류의 창의성을 증폭시키는 도구로서 자리매김할 수 있습니다.

예를 들어, 챗GPT와 같은 대규모 언어 모델은 뛰어난 문장 생성 능력을 바탕으로 소설, 시, 기사, 심지어 코드까지 작성할 수 있습니다. Midjourney, DALL-E 2와 같은 이미지 생성 AI는 텍스트 설명을 기반으로 독창적인 이미지를 만들어내며, 이전에는 상상하기 어려웠던 새로운 시각적 표현을 가능하게 합니다. 이처럼 생성형 AI는 개인의 창의적인 활동을 지원하고, 전문가의 업무 효율성을 높이며, 새로운 사업 기회를 창출하는 데 기여할 수 있습니다.

그러나 생성형 AI의 발전은 윤리적인 문제와 사회적 책임에 대한 심각한 고민을 요구합니다. 생성형 AI가 만들어낸 콘텐츠가 저작권 침해, 허위 정보 유포, 혐오 표현 확산 등 다양한 문제점을 야기할 수 있기 때문입니다. 특히, 이미지 및 비디오 생성 AI 기술이 발전하면서 딥페이크 영상 제작이 쉬워지고, 이는 개인의 명예를 훼손하거나 사회적 혼란을 야기하는 데 악용될 수 있습니다.

따라서, 생성형 AI의 발전과 함께 윤리적 가이드라인 및 규제 마련이 시급합니다. 생성형 AI가 만들어낸 콘텐츠에 대한 출처 표기 의무화, 허위 정보 및 혐오 표현 필터링 기술 개발, AI 윤리 교육 강화 등 다양한 노력이 필요합니다. 또한, 개발자들은 AI 모델 개발 시 윤리적 고려 사항을 반영하고, 사용자들은 AI 기술을 책임감 있게 사용하는 자세를 갖춰야 합니다.

결론적으로, 생성형 AI는 인류의 창의성을 증폭시키고 다양한 분야에서 혁신을 가져올 수 있는 잠재력을 가진 혁신적인 기술입니다. 그러나 동시에 윤리적인 문제와 사회적 책임에 대한 심각한 고민을 요구합니다. 따라서, 생성형 AI의 발전과 함께 윤리적 가이드라인 및 규제 마련, AI 윤리 교육 강화 등 다양한 노력을 통해 창조적 가능성과 책임의 균형을 이루는 것이 중요합니다. 미래 사회에서 생성형 AI가 긍정적인 방향으로 발전하고 인류에게 기여할 수 있도록 우리 모두의 노력이 필요합니다.


------------------------------------------------------------


생성형 AI의 부정적인 측면은 다음과 같습니다.

*   **저작권 침해**: 생성형 AI가 기존 콘텐츠를 학습하여 새로운 콘텐츠를 생성할 때 저작권 침해 문제가 발생할 수 있습니다.
*   **허위 정보 유포**: 생성형 AI가 생성한 콘텐츠가 허위 정보를 포함할 수 있으며, 이는 사회적 혼란을 야기할 수 있습니다.
*   **혐오 표현 확산**: 생성형 AI가 혐오 표현을 생성하거나 확산시키는 데 사용될 수 있습니다.
*   **딥페이크 악용**: 이미지 및 비디오 생성 AI 기술을 이용하여 딥페이크 영상을 쉽게 제작할 수 있으며, 이는 개인의 명예를 훼손하거나 사회적 혼란을 야기하는 데 악용될 수 있습니다.
*   **윤리적 문제 및 사회적 책임**: 생성형 AI의 발전은 윤리적인 문제와 사회적 책임에 대한 심각한 고민을 요구합니다.

------------------------------------------------------------


제공된 텍스트에서 설명하는 생성형 AI의 긍정적인 측면은 다음과 같습니다.

*   **창의성 증폭:** 텍스트, 이미지, 오디오, 비디오 등 다양한 형태의 콘텐츠를 스스로 생성하여 인간의 창의적인 활동을 지원하고 증폭시킬 수 있습니다.
*   **혁신 잠재력:** 예술, 디자인, 콘텐츠 제작, 연구 개발 등 다양한 분야에서 혁신을 가져올 가능성을 제시합니다.
*   **업무 효율성 향상:** 전문가의 업무 효율성을 높여줍니다.
*   **새로운 사업 기회 창출:** 새로운 사업 기회를 창출하는 데 기여할 수 있습니다.
*   **새로운 시각적 표현 가능:** 이전에는 상상하기 어려웠던 새로운 시각적 표현을 가능하게 합니다.
*   **개인의 창작 활동 지원:** 소설, 시, 기사, 코드 등 다양한 형태의 콘텐츠를 생성하여 개인의 창작 활동을 지원합니다.

------------------------------------------------------------


생성형 AI는 텍스트, 이미지, 오디오, 비디오 등 다양한 형태의 콘텐츠를 스스로 만들어내는 혁신적인 기술입니다. 과거에는 데이터를 분석하고 예측하는 데 그쳤던 AI와 달리, 생성형 AI는 데이터 패턴을 학습하여 완전히 새로운 콘텐츠를 창조해낼 수 있다는 점에서 큰 잠재력을 지닙니다.

**긍정적인 측면:**

*   **창의성 증폭:** 인간의 창의적인 활동을 지원하고 증폭시켜 줍니다.
*   **혁신 잠재력:** 예술, 디자인, 콘텐츠 제작, 연구 개발 등 다양한 분야에서 혁신을 가져올 수 있습니다.
*   **업무 효율성 향상:** 전문가의 업무 효율성을 높여줍니다.
*   **새로운 사업 기회 창출:** 새로운 사업 기회를 만들 수 있습니다.
*   **새로운 시각적 표현 가능:** 이전에는 상상하기 어려웠던 새로운 시각적 표현을 가능하게 합니다.
*   **개인의 창작 활동 지원:** 소설, 시, 기사, 코드 등 다양한 형태의 콘텐츠를 생성하여 개인의 창작 활동을 지원합니다.

**부정적인 측면:**

*   **저작권 침해**: 기존 콘텐츠를 학습하여 새로운 콘텐츠를 생성할 때 저작권 침해 문제가 발생할 수 있습니다.
*   **허위 정보 유포**: 허위 정보를 포함한 콘텐츠를 생성하여 사회적 혼란을 야기할 수 있습니다.
*   **혐오 표현 확산**: 혐오 표현을 생성하거나 확산시키는 데 사용될 수 있습니다.
*   **딥페이크 악용**: 이미지 및 비디오 생성 AI 기술을 이용하여 딥페이크 영상을 쉽게 제작하여 개인의 명예를 훼손하거나 사회적 혼란을 야기하는 데 악용될 수 있습니다.
*   **윤리적 문제 및 사회적 책임**: 기술 발전 속도에 비해 윤리적인 기준과 사회적 책임에 대한 논의가 부족합니다.

**결론:**

생성형 AI는 다양한 분야에서 혁신을 가져올 수 있지만, 윤리적인 문제와 사회적 책임에 대한 심각한 고민을 요구합니다. 저작권 침해, 허위 정보 유포, 혐오 표현 확산, 딥페이크 악용 등의 위험을 방지하기 위해 윤리적 가이드라인 및 규제 마련, AI 윤리 교육 강화 등 다양한 노력이 필요합니다. 개발자와 사용자는 모두 책임감을 가지고 기술을 사용해야 합니다. 생성형 AI가 긍정적인 방향으로 발전하고 인류에게 기여할 수 있도록 우리 모두의 노력이 필요합니다.


------------------------------------------------------------


In [ ]:
print(response)


[{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': '생성형 AI의 대해서 설명해주세요.', 'type': 'human', 'id': '130b65e2-a782-4628-8bef-49239630f14f'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': '## 생성형 AI의 발전과 윤리적 고려: 창조적 가능성과 책임의 균형\n\n생성형 AI는 텍스트, 이미지, 오디오, 비디오 등 다양한 형태의 콘텐츠를 스스로 생성해내는 혁신적인 기술입니다. 과거 단순히 데이터를 분석하고 예측하는 데 그쳤던 AI 기술과는 달리, 생성형 AI는 주어진 데이터 패턴을 학습하여 완전히 새로운 콘텐츠를 창조해낼 수 있다는 점에서 엄청난 잠재력을 지니고 있습니다. 이러한 발전은 예술, 디자인, 콘텐츠 제작, 연구 개발 등 다양한 분야에서 혁신을 가져올 가능성을 제시하며, 인류의 창의성을 증폭시키는 도구로서 자리매김할 수 있습니다.\n\n예를 들어, 챗GPT와 같은 대규모 언어 모델은 뛰어난 문장 생성 능력을 바탕으로 소설, 시, 기사, 심지어 코드까지 작성할 수 있습니다. Midjourney, DALL-E 2와 같은 이미지 생성 AI는 텍스트 설명을 기반으로 독창적인 이미지를 만들어내며, 이전에는 상상하기 어려웠던 새로운 시각적 표현을 가능하게 합니다. 이처럼 생성형 AI는 개인의 창의적인 활동을 지원하고, 전문가의 업무 효율성을 높이며, 새로운 사업 기회를 창출하는 데 기여할 수 있습니다.\n\n그러나 생성형 AI의 발전은 윤리적인 문제와 사회적 책임에 대한 심각한 고민을 요구합니다. 생성형 AI가 만들어낸 콘텐츠가 저작권 침해, 허위 정보 유포, 혐오 표현 확산 등 다양한 문

## End of Notebook